In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

## Open GIS Data

In [3]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

## Get Brewery Data

In [4]:
r = requests.get(
    "https://worldpopulationreview.com/state-rankings/beer-production-by-state"
)

In [5]:
soup = BeautifulSoup(r.text, "html.parser")

In [14]:
larger_table = soup.find_all("table")[1]

In [15]:
tr = larger_table.find_all("tr", {"class": "table-row"})[0]
tr.find_all("th")[0].text
float(tr.find_all("td")[3].text)

'4.5M'

In [24]:
tr.find("th").text

'Ohio'

In [25]:
states = []
amounts = []
for tr in larger_table.find_all("tr", {"class": "table-row"}):
    state = tr.find("th").text
    amount = float(tr.find_all("td")[3].text)
    states.append(state)
    amounts.append(amount)

In [29]:
beer_df = pd.DataFrame({"NAME": states, "BEER_G": amounts})
beer_df = beer_df.drop_duplicates()

## Format data and Calculate per 100k

In [30]:
beer_per_state_gdf = states_df[["geometry", "NAME"]].merge(
    beer_df, on="NAME", how="inner"
)

In [32]:
beer_per_state_gdf = beer_per_state_gdf.to_crs(9311)

In [33]:
beer_per_state_gdf.to_file("data/beer.gpkg")